### Predicting Iowa House Prices
This Kaggle project predicted house prices in Ames, Iowa with 79 original features (2006-2010).
The training set had 1460 observations and the test set had 1459 observations. I tried linear regression, Elastic Net, Random Forest and XGBoost, and finally built a weighted average meta model. My predictions ranked top 10% according to Mean Squared Error in November, 2019.

In [ ]:
#import libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm, skew #for some statistics
from sklearn.preprocessing import StandardScaler
from scipy import stats

#import libraries--feature engineering
from sklearn.preprocessing import LabelEncoder
from scipy.special import boxcox1p
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

#import libraries--modeling
from sklearn.linear_model import LinearRegression, ElasticNet, Lasso, Ridge
from sklearn.kernel_ridge import KernelRidge
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import KFold, cross_val_score, train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import os
import warnings
warnings.filterwarnings('ignore')
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn #ignore warnings from sklearn and seaborn

#setup graphs
color = sns.color_palette()
sns.set_style('darkgrid')
%matplotlib inline
pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x)) #Limiting floats output to 3 decimal points

In [ ]:
#import datasets
train = pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')
test = pd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv')
train.shape, test.shape

#### Cleaning Data

In [ ]:
#fix outlier: YrSold is earlier than YrBuilt for the observation 1089 in the test data
test.loc[1089]["YrSold"] = 2009
test.loc[1089]["YrActualAge"] = 0

In [ ]:
#store the 'Id' column then drop it from original datasets--not used in modeling
#axis = 1 indicates col
train_ID = train['Id']
test_ID = test['Id']
train.drop("Id", axis = 1, inplace = True)
test.drop("Id", axis = 1, inplace = True)

In [ ]:
###Outliers
#use a scatter plot to observation the relationshiop between living areas and prices
fig, ax = plt.subplots()
ax.scatter(x = train['GrLivArea'], y = train['SalePrice'])
plt.ylabel('SalePrice', fontsize=13)
plt.xlabel('GrLivArea', fontsize=13)
plt.show()

#Delete outliers in the bottom-right corner of the scatter plot
train = train.drop(train[(train['GrLivArea']>4000) & (train['SalePrice']<300000)].index)

#Check distribution again
fig, ax = plt.subplots()
ax.scatter(train['GrLivArea'], train['SalePrice'])
plt.ylabel('SalePrice', fontsize=13)
plt.xlabel('GrLivArea', fontsize=13)
plt.show()

In [ ]:
#Target variable
#Check the distribution of target variable: saleprice
sns.distplot(train['SalePrice'] , fit=norm);

#Get the fitted parameters used by the function
(mu, sigma) = norm.fit(train['SalePrice'])
print( '\n mu = {:.2f} and sigma = {:.2f}\n'.format(mu, sigma))

#Plot the distribution of salesprice
plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)],
            loc='best')
plt.ylabel('Frequency')
plt.title('SalePrice distribution')

# Label is right-skewed. necessary to perform log transformation to make it more normally distributed.
#use the numpy fuction log1p to apply log(1+x): plus 1 to avoid -inf
train["SalePrice"] = np.log1p(train["SalePrice"])

#Check the new distribution 
sns.distplot(train['SalePrice'] , fit=norm);

# Get the fitted parameters used by the function
(mu, sigma) = norm.fit(train['SalePrice'])
print( '\n mu = {:.2f} and sigma = {:.2f}\n'.format(mu, sigma))

#Now plot the distribution again
plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)],
            loc='best')
plt.ylabel('Frequency')
plt.title('SalePrice distribution')

In [ ]:
#concatenate train and test data to clean data together
ntrain = train.shape[0]
ntest = test.shape[0]
y_train = train.SalePrice.values
all = pd.concat((train, test)).reset_index(drop=True)
all.drop(['SalePrice'], axis=1, inplace=True)
print("all size is : {}".format(all.shape))

#Assign "None" to missing values accordingly:
for col in ('MSSubClass', 'MasVnrType', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'PoolQC', 'MiscFeature', 'Alley', 'Fence', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond'):
    all[col] = all[col].fillna('None')
#Group by neighborhood and fill in missing value by the median LotFrontage of all the neighborhood in the training set
#median, mean functions are not affected by missing values. first, obtain the median of training data
nbh_lot = train.groupby(train.Neighborhood)[['LotFrontage']].median()
#med_lot = neigh_lot.groupby("Neighborhood")["LotFrontage"].transform("median")
#all["LotFrontage"] = all["LotFrontage"].fillna(med_lot)
#all.loc[all.Neighborhood.isin(neigh_lot.Neighborhood), ['LotFrontage']] = neigh_lot['LotFrontage']
all = all.merge(nbh_lot, on=["Neighborhood"], how='left', suffixes=('','_'))
all['LotFrontage'] = all['LotFrontage'].fillna(all['LotFrontage_']).astype(int)
all = all.drop('LotFrontage_', axis=1)

#all["LotFrontage"] = train.groupby("Neighborhood")["LotFrontage"].transform(lambda x: x.fillna(x.median()))
#GarageYrBlt, GarageArea and GarageCars : Replacing missing data with 0 (Since No garage = no cars in such garage.)
#BsmtFinSF1, BsmtFinSF2, BsmtUnfSF, TotalBsmtSF, BsmtFullBath and BsmtHalfBath : missing values are likely zero for having no basement
for col in ('MasVnrArea', 'GarageYrBlt', 'GarageArea', 'GarageCars', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF','TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath'):
    all[col] = all[col].fillna(0)
#Remove "Utilities"-- For this categorical feature all records are "AllPub", except for one "NoSeWa" and 2 NA .
all = all.drop(['Utilities'], axis=1)
#Functional : data description says NA means typical
all["Functional"] = all["Functional"].fillna("Typ")
#vars with only one NA value, use mode of this var in the training set to prevent data leakage
for col in ('KitchenQual', 'Electrical', 'Exterior1st', 'Exterior2nd', 'MSZoning', 'SaleType'):
    all[col] = all[col].fillna(train[col].mode()[0])

#check if there is any remaining missing values
all[all.isna().any(axis=1)]

#### Feature Engineering

In [ ]:
#Changing OverallCond into a categorical variable
all['OverallCond'] = all['OverallCond'].apply(str)

#Year and month sold are transformed into categorical features.
all['YrSold'] = all['YrSold'].astype(str)
all['MoSold'] = all['MoSold'].astype(str)

#LabelEncode categorical variables with orders: different numbers in the same col
cates = ('FireplaceQu', 'BsmtQual', 'BsmtCond', 'GarageQual', 'GarageCond', 
        'ExterQual', 'ExterCond','HeatingQC', 'PoolQC', 'KitchenQual', 'BsmtFinType1', 
        'BsmtFinType2', 'Functional', 'Fence', 'BsmtExposure', 'GarageFinish', 'LandSlope',
        'LotShape', 'PavedDrive', 'Street', 'Alley', 'CentralAir', 'MSSubClass', 'OverallCond', 
        'YrSold', 'MoSold')
# process columns, apply LabelEncoder to categorical features
for c in cates:
    lbl = LabelEncoder() 
    lbl.fit(list(all[c].values)) 
    all[c] = lbl.transform(list(all[c].values))

#add a feature: total areas
all['TotalSF'] = all['TotalBsmtSF'] + all['1stFlrSF'] + all['2ndFlrSF']

#use one hot code transfer categorical values -- preparing for PCA
all = pd.get_dummies(all)
print(all.shape)


#clean Skewed features
num_vars = all.dtypes[all.dtypes != "object"].index

# Check the skew of all numerical features
skewed_vars = all[num_vars].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
print("\nSkewness of numerical features: \n")
skewness = pd.DataFrame({'Skew' :skewed_vars})
skewness.head(15)

# apply Box Cox Transformation to (highly) skewed features
skewness = skewness[abs(skewness) > 0.8]
skewed_features = skewness.index
lam = 0.25
for f in skewed_features:
    all[f] = boxcox1p(all[f], lam)

In [ ]:
#separate train and test data again
train = all[:ntrain]
test = all[ntrain:]

In [ ]:
#standardize dataset -- preparing for PCA: too many features
scaler = StandardScaler()
#fit on training set only.
scaler.fit(train)
# Apply transform to both the training set and the test set.
train = scaler.transform(train)
test = scaler.transform(test)

#pca: reduction dimensionality of features
#make an instance of the Model: scikit-learn choose the minimum number of principal components such that 95% of the variance is retained
pca = PCA(.95)
pca.fit(train)
train = pca.transform(train)
test = pca.transform(test)

#### Modeling

In [ ]:
#try a simple linear regression first
metric = 'neg_mean_squared_error'
kfold = KFold(n_splits=10, shuffle=True, random_state=1)
print(f"{np.sqrt(-cross_val_score(LinearRegression(), train, y_train, cv=kfold, scoring=metric)).mean():.4f} Error")

#Grid search of elastic net, random forest and xgb
#LightGBM takes forever to run
#KernelRidge generates errors: Mean Squared Logarithmic Error cannot be used when targets contain negative values
#ElasticNet
elastic = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0004, l1_ratio=1, random_state=1))
param_grid = {
    'elasticnet__alpha' : np.linspace(0.0001, 0.001, 10),
    'elasticnet__l1_ratio' : np.linspace(0.2, 2, 20),
}
search = GridSearchCV(elastic, param_grid, cv=10, scoring=metric, n_jobs=-1)
search.fit(train, y_train)
best_params_ela = search.best_params_
print(f"{search.best_params_}")
print(f"{np.sqrt(-search.best_score_):.4}")

In [ ]:
#random forest
rdf = make_pipeline(RobustScaler(), RandomForestRegressor(max_depth=4, n_estimators=100, random_state=1))
param_grid={
            'max_depth': range(2,4),
            'n_estimators': (50, 100),
        }
search = GridSearchCV(RandomForestRegressor(), param_grid, cv=10, scoring=metric, n_jobs=-1)   
search.fit(train, y_train)
best_params_forest = search.best_params_
print(f"{search.best_params_}")
print(f"{np.sqrt(-search.best_score_):.4}")

In [ ]:
#XGBboost
xgbreg = xgb.XGBRegressor(objective="reg:squarederror",
                             colsample_bytree=0.45, gamma=0.046, 
                             learning_rate=0.03, max_depth=2, 
                             min_child_weight=0.4, n_estimators=10,
                             reg_alpha=0.47, reg_lambda=0.8,
                             subsample=0.5, random_state=1, n_jobs=-1)

param_grid = {
    'xgb__max_depth' : [2, 3],
    'xgb__estimators' : [10, 25, 50],
    "xgb__learning_rate" : [0.01, 0.02],
    "xgb__min_child_weight" : [0.2, 0.3],
    }
search = GridSearchCV(xgbreg, param_grid, cv=3, scoring=metric, n_jobs=-1)
search.fit(train, y_train)
best_params_xgb = search.best_params_
print(f"{search.best_params_}")
print(f"{np.sqrt(-search.best_score_):.4}")

In [ ]:
#elastic net
elastic_net = make_pipeline(RobustScaler(), ElasticNet(alpha=best_params_ela['elasticnet__alpha'], 
                                                   l1_ratio=best_params_ela['elasticnet__l1_ratio'], random_state=1))
forest = make_pipeline(RobustScaler(), RandomForestRegressor(max_depth=best_params_forest["max_depth"], n_estimators=best_params_forest["n_estimators"],  random_state=1))
xgb_reg = xgb.XGBRegressor(objective="reg:squarederror",
                             colsample_bytree=0.45, gamma=0.046, 
                             max_depth=best_params_xgb["xgb__max_depth"], n_estimators=best_params_xgb["xgb__estimators"], 
                             learning_rate = best_params_xgb['xgb__learning_rate'], min_child_weight= best_params_xgb["xgb__min_child_weight"],
                             reg_alpha=0.47, reg_lambda=0.8,
                             subsample=0.5, random_state=1, n_jobs=-1)

In [ ]:
#create a meta model
classifiers = [elastic_net, forest, xgb_reg]
clf_names   = ["elastic_net", "random_forest", "XGBboost"]
weights = [0.8, 0.15, 0.05]

In [ ]:
#make predictions
predictions_exp= []
for clf_name, clf, weight in zip(clf_names, classifiers, weights):
    print(f"{clf_name} {np.sqrt(-cross_val_score(clf, train, y_train, scoring=metric).mean()):.5f}")
    clf.fit(train, y_train)
    preds = clf.predict(test)
    predictions_exp.append(weight*np.expm1(preds))
prediction_final = pd.DataFrame(predictions_exp).sum().T.values